In [1]:
import os

In [2]:
%pwd

'd:\\technical\\AI-ML\\ml_projects\\text-summarizer-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\technical\\AI-ML\\ml_projects\\text-summarizer-project'

#### 1. update entity

In [81]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

#### 2. update the configuration manager in src config

In [82]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [83]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath: CONFIG_FILE_PATH, # type: ignore
                 params_filepath: PARAMS_FILE_PATH): # type: ignore
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])  

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config =  DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

#### 3. update the components

In [84]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [85]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 

#### 4. upadte pipeline

In [86]:
from pathlib import Path

# Convert string paths to Path objects
# config_filepath = Path("config\config.yaml")  # Replace with actual path
# params_filepath = Path("params.yaml")
try:
    config = ConfigurationManager(config_filepath=config_filepath, params_filepath=params_filepath)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-29 18:09:58,913: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-29 18:09:58,915: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-29 18:09:58,916: INFO: common: created directory at: artifacts]
[2024-10-29 18:09:58,918: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-29 18:10:00,330: INFO: 309289457: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FEC7:2EBA62:3F2FBE:4FD5CD:6720C950
Accept-Ranges: bytes
Date: Tue, 29 Oct 2024 12:39:58 GMT
Via: 1.1 varnish
X-Served-By: cache-del21750-DEL
X-Cache: HI